In [1]:
pip install jsonpath

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import base64
import json
import sys
import time
import pymysql
import boto3
import jsonpath 

In [9]:
def get_headers(client_id, client_secret):
    endpoint = 'https://accounts.spotify.com/api/token'
    encoded = base64.b64encode((client_id+':'+client_secret).encode('utf-8')).decode('ascii')
    headers = {'Authorization': 'Basic {auth}'.format(auth=encoded)}
    payload = {'grant_type':'client_credentials'}
    try:
        r = requests.post(endpoint, data=payload, headers=headers)
    except:
        print('인증오류가 발생되었습니다!')
        sys.exit(1)
    if r.status_code!=200:
        if r.status_code==401:
            print('Client_id와 Secret이 잘못되었습니다!')
            sys.exit(1)
        else:
            print('인증오류가 발생되었습니다!:', r.status_code)
            sys.exit(1)
    acess_token = json.loads(r.text).get('access_token')
    headers = {'Authorization': 'Bearer {auth}'.format(auth=acess_token)}
    return headers

In [10]:
def get_ids():
    host = "hyundb.cqgrmpcsglic.ap-northeast-2.rds.amazonaws.com"
    port = 3306
    username = 'hyunjilee'
    database = 'production'
    pw = ''
    conn = pymysql.connect(host=host, user=username, passwd=pw, db=database,
                           port=port, use_unicode=True, charset='utf8')
    cursor = conn.cursor()
    cursor.execute('SELECT artist_id FROM artists')
    ids = [i for (i,) in cursor.fetchall()]
    conn.commit()
    cursor.close()
    conn.close()
    return ids

In [12]:
def top_tracks(m):
    headers = get_headers(client_id, client_secret)
    params = {'market':'KR'}
    top_tracks_keys = {'track_id':['S','id'],
                       'track_name':['S','name'],
                       'popularity': ['N','popularity'],
                       'external_url' : ['S','external_urls.spotify']}
    url = 'https://api.spotify.com/v1/artists/{a_id}/top-tracks'.format(a_id=m)
    try:
        res = requests.get(url, params=params, headers=headers)
    except:
        print('top-tracks획득에 오류가 발생되었습니다!')
        sys.exit(1)
    if res.status_code!=200:
        if res.status_code==401:
            headers = get_headers(client_id, client_secret)
            res = requests.get(endpoint, params=params, headers=headers)
        elif res.status_code==429:
            retry_time = json.loads(res.headers)['Retry-After']
            time.sleep(int(retry_time) + 1)
            res = requests.get(endpoint, params=params, headers=headers)
        else:
            print('top-tracks획득에 오류가 발생되었습니다!',res.status_code)
            sys.exit(1)
    raw = json.loads(res.text)
    top_tracks = []
    for t in raw['tracks']:
        data = {}
        data.update({'artist_id':{'S':m}})
        for k, v in top_tracks_keys.items():
            ct = jsonpath.jsonpath(t, v[1])[0]
            if ct==False:
                if v[1]=='popularity':
                    ct = 0
                else:
                    ct = 'UNK'
            data.update({k: {v[0]:str(ct)}})
        top_tracks.append(data)
    return top_tracks

In [16]:
client_id = '77ba7a249d9940b79c687b05f518ebec'
client_secret = ''
# my_ids = get_ids()

In [38]:
headers = get_headers(client_id, client_secret)
params = {'market':'KR'}
top_tracks_keys = {'track_id':['S','id'],
                   'track_name':['S','name'],
                   'popularity': ['N','popularity'],
                   'external_url' : ['S','external_urls.spotify']}
url = 'https://api.spotify.com/v1/artists/{a_id}/top-tracks'.format(a_id=m)
try:
    res = requests.get(url, params=params, headers=headers)
except:
    print('top-tracks획득에 오류가 발생되었습니다!')
    sys.exit(1)
if res.status_code!=200:
    if res.status_code==401:
        headers = get_headers(client_id, client_secret)
        res = requests.get(endpoint, params=params, headers=headers)
    elif res.status_code==429:
        retry_time = json.loads(res.headers)['Retry-After']
        time.sleep(int(retry_time) + 1)
        res = requests.get(endpoint, params=params, headers=headers)
    else:
        print('top-tracks획득에 오류가 발생되었습니다!',res.status_code)
        sys.exit(1)
raw = json.loads(res.text)
top_tracks = []
for t in raw['tracks']:
    data = {}
    data.update({'artist_id':{'S':m}})
    for k, v in top_tracks_keys.items():
        ct = jsonpath.jsonpath(t, v[1])[0] #t에서 v[1] 에 해당하는 것들을뽑아라
        if ct==False:
            if v[1]=='popularity':
                ct = 0
            else:
                ct = 'UNK'
        data.update({k: {v[0]:str(ct)}})
        print(data)
    top_tracks.append(data)

{'artist_id': {'S': '7zICaxnDB9ZprDSiFpvbbW'}, 'track_id': {'S': '1Xunvmk47Mju6oZlgzm2Ty'}}
{'artist_id': {'S': '7zICaxnDB9ZprDSiFpvbbW'}, 'track_id': {'S': '1Xunvmk47Mju6oZlgzm2Ty'}, 'track_name': {'S': 'Burbons And Lacs'}}
{'artist_id': {'S': '7zICaxnDB9ZprDSiFpvbbW'}, 'track_id': {'S': '1Xunvmk47Mju6oZlgzm2Ty'}, 'track_name': {'S': 'Burbons And Lacs'}, 'popularity': {'N': '58'}}
{'artist_id': {'S': '7zICaxnDB9ZprDSiFpvbbW'}, 'track_id': {'S': '1Xunvmk47Mju6oZlgzm2Ty'}, 'track_name': {'S': 'Burbons And Lacs'}, 'popularity': {'N': '58'}, 'external_url': {'S': 'https://open.spotify.com/track/1Xunvmk47Mju6oZlgzm2Ty'}}
{'artist_id': {'S': '7zICaxnDB9ZprDSiFpvbbW'}, 'track_id': {'S': '0l3wp8iEtN8rgag9eTeorW'}}
{'artist_id': {'S': '7zICaxnDB9ZprDSiFpvbbW'}, 'track_id': {'S': '0l3wp8iEtN8rgag9eTeorW'}, 'track_name': {'S': "Make 'Em Say Ugh"}}
{'artist_id': {'S': '7zICaxnDB9ZprDSiFpvbbW'}, 'track_id': {'S': '0l3wp8iEtN8rgag9eTeorW'}, 'track_name': {'S': "Make 'Em Say Ugh"}, 'popularity': {'N

In [15]:
my_tops = []
for m in my_ids:
    my_tops.extend(top_tracks(m))
    
seq = 20 # max=25
my_tops = [my_tops[i:i+seq] for i in range(0, len(my_tops), seq)]

In [8]:
try:
    dynamodb = boto3.client('dynamodb',
                             region_name='ap-northeast-2',
                             endpoint_url='http://dynamodb.ap-northeast-2.amazonaws.com')
except:
    print('DynamoDB접속오류!')
    sys.exit(1)

for my_top in my_tops:
    my_res = {'top_tracks': [{'PutRequest': {'Item' : m}} for m in my_top]}
    dynamodb.batch_write_item(RequestItems=my_res)
    time.sleep(5)